# Read SWMM result files

In this notebook we give examples how to read SWMM result files **out**. Most of the functionality is the same as reading res1d files, which is described in [res1d.ipynb](res1d.ipynb).

In [ ]:
from mikeio1d import Res1D
from mikeio1d.query import QueryDataNode, QueryDataReach, QueryDataCatchment

## SWMM out file

Read entire res file into a data frame. Here we use results from a standard **Example1.inp** distributed together with EPA SWMM 5.1.

In [ ]:
file_path = "../tests/testdata/swmm.out"
res = Res1D(file_path)
df = res.read()

Let's peek at the full data frame coming from EPANET result.

In [ ]:
df.head()

For SWMM there quite a lot of available quantities:

In [ ]:
res.quantities

Here we retrieve flow in a pipe with ID **10**, which has connecting junctions **17** and **18**. Note that in order to have network connectivity the corresponding setup **inp** file is needed, which has to be called the same as the result file, i.e., for **swmm.out** we need to have **Swmm.inp** with the relevant network information.

In [ ]:
pipe_id = '10'
pipe_data_item = res.reaches[pipe_id]
node_start = res.data.Nodes[pipe_data_item.StartNodeIndex]
node_end = res.data.Nodes[pipe_data_item.EndNodeIndex]
print(f'Start node ID: {node_start.ID}')
print(f'End node ID: {node_end.ID}')

In [ ]:
df[['SWMM_LINK_FLOW:10', 'SWMM_NODE_DEPTH:17', 'SWMM_NODE_DEPTH:18']].head()

Let's plot the above results, but this time we will retrieve the data using **QueryDataReach** and **QueryDataNode**. 

In [ ]:
pipe_query = QueryDataReach('SWMM_LINK_FLOW', pipe_id)
df_pipe = res.read(queries=[pipe_query])
df_pipe.plot();

node_start_query = QueryDataNode('SWMM_NODE_DEPTH', node_start.ID)
node_end_query = QueryDataNode('SWMM_NODE_DEPTH', node_end.ID)
df_nodes = res.read(queries=[node_start_query, node_end_query])
df_nodes.plot();

Finally, the catchment results are also stored in SWMM out file. The current result file has the following catchments:

In [ ]:
res.catchments

Available quantities for catchments can be obtained from one of the catchments (for example, `ID='5'`) as

In [ ]:
catchment_id = '5'
swmm_catchment = res.catchments[catchment_id]
catchment_quantities = [data_item.Quantity.Id for data_item in swmm_catchment.DataItems]
print(catchment_quantities)

We can create quantity `'SWMM_SUBCATCH_RUNOFF'` queries for all catchments as

In [ ]:
catchment_queries = [QueryDataCatchment('SWMM_SUBCATCH_RUNOFF', catchment_id) for catchment_id in res.catchments]
df_catchments = res.read(queries=catchment_queries)

In [ ]:
df_catchments.head()

Let's plot the runoff for catchment `ID='5'`

In [ ]:
df_catchments['SWMM_SUBCATCH_RUNOFF:5'].plot();